# 110. Basic Linear Regression

In [1]:
import numpy as np
import torch

# 1) Feed Forward 구현

- weight와 bias를 알고 있다고 가정하고 Feed Forward 함수를 직접 구현  
- 다음의 선형회귀 문제를 pytorch 로 해결  

$$b=-1,w=2$$

$$\hat{y}=-1+2x$$

In [2]:
w = torch.tensor(2.0)
b = torch.tensor(-1.0)
print(w, b)

tensor(2.) tensor(-1.)


In [3]:
def forward(x):
    yhat = w * x + b
    return yhat

x = 1 일 때 `y = 2x - 1`이므로 y 는 1

In [4]:
x = torch.tensor([1.0])
yhat = forward(x)

print(f"x = {x.item()} 일 때, y = {yhat.item()}")

x = 1.0 일 때, y = 1.0


x 가 multiple inputs (array) 인 경우:

In [5]:
x = torch.tensor([[1.0], [2.0], [3.0]])
forward(x)

tensor([[1.],
        [3.],
        [5.]])

In [6]:
y_hat = forward(x)
y_hat

tensor([[1.],
        [3.],
        [5.]])

<h1>2) torch.nn 의 Linear 클래스를 이용한 Linear Regression</h1>

In [24]:
from torch.nn import Linear
torch.manual_seed(1)

model = Linear(in_features=1, out_features=1, bias=True)

<code>state_dict()</code> method 는 각 layer 의 parameter tensor 를 dictionary 로 반환하므로 직접 값을 update 할 수 있다.

In [25]:
print(model.state_dict())
print()
print(model.state_dict()['weight'], model.state_dict()['bias'])
print()
print(model.weight)
print(model.bias)

OrderedDict([('weight', tensor([[0.5153]])), ('bias', tensor([-0.4414]))])

tensor([[0.5153]]) tensor([-0.4414])

Parameter containing:
tensor([[0.5153]], requires_grad=True)
Parameter containing:
tensor([-0.4414], requires_grad=True)


In [26]:
model.state_dict()['weight'][0] = torch.tensor(2)
model.state_dict()['bias'][0]     = torch.tensor(-1)

In [27]:
model.state_dict()

OrderedDict([('weight', tensor([[2.]])), ('bias', tensor([-1.]))])

In [28]:
# x = [[1.0]]일 경우 prediction
x = torch.tensor([[1.0]])
yhat = model(x)
yhat.item()

1.0

In [29]:
# x 가 multiple inputs (array) 인 경우:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
print(yhat.detach().numpy())

[[1.]
 [3.]
 [5.]]


<h1>3) 사용자 정의 Module 작성</h1>

In [30]:
from torch import nn

class LR(nn.Module):
    def __init__(self, input_size, output_size):
        super(LR, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [31]:
model = LR(1, 1)

model.state_dict()

OrderedDict([('linear.weight', tensor([[-0.1939]])),
             ('linear.bias', tensor([0.4694]))])

In [32]:
model.state_dict()['linear.weight'][0] = torch.tensor(2)
model.state_dict()['linear.bias'][0]     = torch.tensor(-1)

print(list(model.parameters()))
print()
print(model.linear)

[Parameter containing:
tensor([[2.]], requires_grad=True), Parameter containing:
tensor([-1.], requires_grad=True)]

Linear(in_features=1, out_features=1, bias=True)


In [33]:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
print(yhat.detach().numpy())

[[1.]
 [3.]
 [5.]]


In [34]:
list(model.parameters())

[Parameter containing:
 tensor([[2.]], requires_grad=True),
 Parameter containing:
 tensor([-1.], requires_grad=True)]

In [35]:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
yhat

tensor([[1.],
        [3.],
        [5.]], grad_fn=<AddmmBackward0>)

In [36]:
yhat.detach().numpy()

array([[1.],
       [3.],
       [5.]], dtype=float32)

# Backpropagation 과 경사하강법 적용

- weight 와 bias를 스스로 학습
- (온도, 강수량, 습도)를 feature 로 입력 받아  (사과, 오렌지)의 수확량(label)을 선형회귀로 예측하는 model 작성

## Data

In [17]:
# Input (습도, 강수량, 최고온도, 최저온도)
inputs = np.array([
                   [73, 67, 43, 10], 
                   [91, 88, 64, 5], 
                   [87, 134, 58, 2], 
                   [102, 43, 37, 4], 
                   [69, 96, 70, 5]], dtype='float32')

In [18]:
# Targets - (apples, oranges) 수확량
targets = np.array([
                    [56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

- input 과 target을 tensor로 변환

In [19]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs.size())
print(targets.size())

torch.Size([5, 4])
torch.Size([5, 2])


# 1) torch.nn Linear Class 직접 사용

In [20]:
from torch.nn import Linear
from torch import optim

torch.manual_seed(1)

In [21]:
model = Linear(in_features=4, out_features=2)

In [22]:
criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [23]:
for epoch in range(100):
    for x, y in zip(inputs, targets):
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [24]:
preds = model(inputs)
loss = criterion(preds, targets)
print(loss)

tensor(4.0758, grad_fn=<MseLossBackward0>)


In [25]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 58.021572  69.74707 ]
 [ 83.15649   98.7855  ]
 [119.965004 128.4409  ]
 [ 21.83963   34.988026]
 [102.8034   117.893616]]


# 2) 사용자 정의 model 사용

- Linear Regression 사용자 정의 model 

In [26]:
class LinearReg(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearReg, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        yhat = self.fc(x)
        return yhat

In [27]:
model = LinearReg(inputs.shape[1], targets.shape[1])

In [28]:
criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [29]:
print(model)
print(model.state_dict())

LinearReg(
  (fc): Linear(in_features=4, out_features=2, bias=True)
)
OrderedDict([('fc.weight', tensor([[ 0.1387,  0.0247,  0.1826, -0.1949],
        [-0.0365, -0.0450,  0.0725, -0.0020]])), ('fc.bias', tensor([0.4371, 0.1556]))])


In [30]:
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.0001,
   'momentum': 0,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'maximize': False,
   'params': [0, 1]}]}

## Train the model

In [31]:
for epoch in range(100):
    for x, y in zip(inputs, targets):
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [32]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3590,  0.8959,  0.6336, -0.3657],
         [-0.3081,  0.7807,  0.9301, -0.0332]], requires_grad=True),
 Parameter containing:
 tensor([0.4331, 0.1568], requires_grad=True)]

In [33]:
model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3590,  0.8959,  0.6336, -0.3657],
                      [-0.3081,  0.7807,  0.9301, -0.0332]])),
             ('fc.bias', tensor([0.4331, 0.1568]))])

In [34]:
preds = model(inputs)

In [35]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 57.842937  69.6347  ]
 [ 85.32951  100.18176 ]
 [125.2741   131.84752 ]
 [ 24.323542  36.581146]
 [104.19539  118.78626 ]]


## Save and Load Model

In [36]:
# model save
torch.save(model.state_dict(), 'model.pt')

model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3590,  0.8959,  0.6336, -0.3657],
                      [-0.3081,  0.7807,  0.9301, -0.0332]])),
             ('fc.bias', tensor([0.4331, 0.1568]))])

- model reload 및 parameter 복원 확인

In [37]:
reloaded = LinearReg(inputs.shape[1], targets.shape[1])
reloaded.load_state_dict(torch.load('model.pt'))

reloaded.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3590,  0.8959,  0.6336, -0.3657],
                      [-0.3081,  0.7807,  0.9301, -0.0332]])),
             ('fc.bias', tensor([0.4331, 0.1568]))])

In [38]:
preds = reloaded(inputs)

In [39]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 57.842937  69.6347  ]
 [ 85.32951  100.18176 ]
 [125.2741   131.84752 ]
 [ 24.323542  36.581146]
 [104.19539  118.78626 ]]
